## Setting Up Your Python Environment

In [1]:
# %%capture
# # Install PyTorch with CUDA
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# # Install additional dependencies
# !pip install pandas nobuco tensorflowjs

# # Install utility packages
# !pip install cjm_yolox_pytorch

## Importing the Required Dependencies

In [2]:
# Import Python Standard Library dependencies
import json
from pathlib import Path

# Import the pandas package
import pandas as pd

# Import timm library
import timm

# Import PyTorch dependencies
import torch
from torch import nn

# Import Nobuco dependencies
from nobuco import pytorch_to_keras, ChannelOrder

# Import TensorFlow
import tensorflow as tf

# Import TensorFlow.js dependencies
from tensorflowjs import converters, quantization

2023-09-26 12:01:18.984731: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-26 12:01:19.010130: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-26 12:01:19.553434: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Setting Up the Project

### Set the Directory Paths

In [3]:
# The name for the project
project_name = f"pytorch-timm-image-classifier"

# The path for the project folder
project_dir = Path(f"./{project_name}/")

# Create the project directory if it does not already exist
project_dir.mkdir(parents=True, exist_ok=True)

# The path to the checkpoint folder
checkpoint_dir = Path(project_dir/f"2023-08-12_15-21-16")

pd.Series({
    "Project Directory:": project_dir,
    "Checkpoint Directory:": checkpoint_dir,
}).to_frame().style.hide(axis='columns')

Project Directory:,pytorch-timm-image-classifier
Checkpoint Directory:,pytorch-timm-image-classifier/2023-08-12_15-21-16


## Loading the Checkpoint Data

### Load the Class Labels

In [4]:
# The class labels path
class_labels_path = list(checkpoint_dir.glob('*classes.json'))[0]

# Load the JSON class labels data
with open(class_labels_path, 'r') as file:
        class_labels_json = json.load(file)

# Get the list of classes
class_names = class_labels_json['classes']

# Print the list of classes
pd.DataFrame(class_names)

,0
0,call
1,dislike
2,fist
3,four
4,like
5,mute
6,no_gesture
7,ok
8,one
9,palm


### Load the Model Checkpoint

In [5]:
# The model checkpoint path
checkpoint_path = list(checkpoint_dir.glob('*.pth'))[0]

# Load the model checkpoint onto the CPU
model_checkpoint = torch.load(checkpoint_path, map_location='cpu')

### Load the Finetuned Model

In [6]:
# Specify the model configuration
model_type = checkpoint_path.stem

# Create a model with the number of output classes equal to the number of class names
model = timm.create_model(model_type, num_classes=len(class_names))

# Initialize the model with the checkpoint parameters and buffers
model.load_state_dict(model_checkpoint)

<All keys matched successfully>

### Get the Normalization Stats

In [7]:
# Import the resnet module
from timm.models import resnet

# Get the default configuration of the chosen model
model_cfg = resnet.default_cfgs[model_type].default.to_dict()

# Retrieve normalization statistics (mean and std) specific to the pretrained model
mean, std = model_cfg['mean'], model_cfg['std']
norm_stats = (mean, std)
norm_stats

((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))

## Converting the Model to TensorFlow

### Prepare the Model for Inference

#### Define model export wrapper

In [8]:
class InferenceWrapper(nn.Module):
    def __init__(self, model, normalize_mean, normalize_std, scale_inp=False, channels_last=False):
        super().__init__()
        self.model = model
        self.register_buffer("normalize_mean", normalize_mean)
        self.register_buffer("normalize_std", normalize_std)
        self.scale_inp = scale_inp
        self.channels_last = channels_last
        self.softmax = nn.Softmax(dim=1)

    def preprocess_input(self, x):
        if self.scale_inp:
            x = x / 255.0

        if self.channels_last:
            x = x.permute(0, 3, 1, 2)

        x = (x - self.normalize_mean) / self.normalize_std
        return x

    def forward(self, x):
        x = self.preprocess_input(x)
        x = self.model(x)
        x = self.softmax(x)
        return x

#### Wrap model with preprocessing and post-processing steps

In [9]:
# Define the normalization mean and standard deviation
mean_tensor = torch.tensor(norm_stats[0]).view(1, 3, 1, 1)
std_tensor = torch.tensor(norm_stats[1]).view(1, 3, 1, 1)

# Set the model to evaluation mode
model.eval();

# Wrap the model with preprocessing and post-processing steps
wrapped_model = InferenceWrapper(model, 
                                 mean_tensor, 
                                 std_tensor, 
                                 scale_inp=True, # Scale input values from the rang [0,255] to [0,1]
                                 channels_last=True, # Have the model expect input in channels-last format
                                )

### Prepare the Input Tensor

In [10]:
input_tensor = torch.randn(1, 256, 256, 3)

### Convert the PyTorch Model to Keras

In [11]:
keras_model = pytorch_to_keras(
    wrapped_model, 
    args=[input_tensor],
    inputs_channel_order=ChannelOrder.PYTORCH,
    outputs_channel_order=ChannelOrder.PYTORCH, 
)

2023-09-26 12:01:20.929978: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-26 12:01:20.930215: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Legend:
    Green — conversion successful
    Yellow — conversion imprecise
    Red — conversion failed
    Red — no converter found
    Bold — conversion applied directly
    * — subgraph reused
    Tensor — this output is not dependent on any of subgraph's input tensors
    Tensor — this input is a parameter / constant
    Tensor — this tensor is useless

InferenceWrapper[__main__](float32_0<1,256,256,3>) -> float32_170<1,19>
 │  __truediv__[torch.Tensor](float32_0<1,256,256,3>, 255.0) -> float32_1<1,256,256,3>
 │  permute[torch.Tensor](float32_1<1,256,256,3>, 0, 3, 1, 2) -> float32_2<1,3,256,256>
 │  __sub__[torch.Tensor](float32_2<1,3,256,256>, float32_3<1,3,1,1>) -> float32_4<1,3,256,256>
 │  __truediv__[torch.Tensor](float32_4<1,3,256,256>, float32_5<1,3,1,1>) -> float32_6<1,3,256,256>
 │  ResNet[timm.models.resnet](float32_6<1,3,256,256>) -> float32_169<1,19>
 │   │  Sequential[torch.nn.modules.container](float32_6<1,3,256,256>) -> float32_22<1,64,128,128>
 │   │   │  Conv2d[tor

## Enabling Dynamic Input Dimensions

### Define New Input Dimensions

In [12]:
# Get the current input shape
input_shape = keras_model.layers[0].input_shape[0][1:]

# Make every dimension except the channel dimension dynamic
dynamic_input_shape = tuple(i if i == 3 else None for i in input_shape)

pd.Series({
    "Source Input Shape:": input_shape,
    "Dynamic Input Shape:": dynamic_input_shape,
}).to_frame().style.hide(axis='columns')

Source Input Shape:,"(256, 256, 3)"
Dynamic Input Shape:,"(None, None, 3)"


### Build Dynamic Keras Model

In [13]:
# Create a Keras tensor with the dynamic input shape
inputs = tf.keras.Input(shape=dynamic_input_shape)
# Get a Keras tensor with the dynamic output shape 
outputs = keras_model(inputs)

# Build a Keras model with dynamic input and output shapes
dynamic_model = tf.keras.Model(inputs, outputs)
# Add the trained weights to the dynamic Keras model
dynamic_model.set_weights(keras_model.get_weights())

### Save the Keras Model in SavedModel format

In [14]:
# Set the folder path for the SavedModel files
savedmodel_dir = Path(f"{checkpoint_dir}/{class_labels_path.stem.removesuffix('-classes')}-{model_type}-tf")
# Save the TensorFlow model to disk
dynamic_model.save(savedmodel_dir, save_format="tf")

INFO:tensorflow:Assets written to: pytorch-timm-image-classifier/2023-08-12_15-21-16/hagrid-classification-512p-no-gesture-150k-zip-resnet18d-tf/assets


INFO:tensorflow:Assets written to: pytorch-timm-image-classifier/2023-08-12_15-21-16/hagrid-classification-512p-no-gesture-150k-zip-resnet18d-tf/assets


## Exporting the Model to TensorFlow.js

In [15]:
# Set the path for TensorFlow.js model files
tfjs_model_dir = f"{savedmodel_dir}js-uint8"

# Convert the TensorFlow SavedModel to a TensorFlow.js Graph model
converters.convert_tf_saved_model(saved_model_dir=str(savedmodel_dir), 
                                  output_dir=tfjs_model_dir, 
                                  quantization_dtype_map={quantization.QUANTIZATION_DTYPE_UINT8:True}
                                 )

2023-09-26 12:01:34.786450: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-26 12:01:34.786548: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2023-09-26 12:01:34.786623: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2023-09-26 12:01:34.786868: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-26 12:01:34.786922: W tensorflow/core/common_runtime/gpu/gpu_d